<a href="https://colab.research.google.com/github/ManiChauras/HPC_MPI_Project/blob/main/Project_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install mpi4py
!which mpiexec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746500 sha256=5893bf64907886185abe65acc41e51d4deed50b53ea76f94457d542ab5cbbe88
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py
/usr/bin/mpiexec


In [ ]:
# Modified
%%writefile MPI.py
import numpy as np
import copy
import time
# from tqdm import tqdm
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
procs = comm.Get_size()



Nx = 128
k = Nx//procs
Ny = 128
Nz = 128

Lx = 12
Ly = 12
Lz = 12
tmax = 0.005
dt = 0.001
g = 0

nstep = int(tmax/dt)

if Nx != 1 and Ny == 1 and Nz == 1:
    dimension = 1
elif Nx != 1 and Ny != 1 and Nz == 1:
    dimension = 2
elif Nx != 1 and Ny != 1 and Nz != 1:
    dimension = 3

dx = Lx/Nx
dy = Ly/Ny
dz = Lz/Nz


x = np.arange(-Nx//2+1 + rank*k, -Nx//2+1 + (rank+1)*k) * dx

volume = dx

if dimension == 2:
    y=np.arange(-Ny//2+1, Ny//2+1) * dy
    volume = dx * dy
    x_mesh, y_mesh = np.meshgrid(x, y, indexing='ij')

elif dimension == 3:
    y=np.arange(-Ny//2+1, Ny//2+1) * dy
    z = np.arange(-Nz//2+1, Nz//2+1) * dz
    volume = dx * dy * dz
    x_mesh, y_mesh, z_mesh = np.meshgrid(x, y, z, indexing='ij')


def set_initialcond():
    if dimension==1:
        '''
        Initial wavefunction
        '''
        wfc = np.zeros((k+2), dtype = np.complex128)
        wfc[1:-1] = ((1/np.pi)**(1/4))*np.exp(-(x**2)/2)+0j    #1D initial condition(shape 128)
        '''
        potential
        '''
        pot = np.zeros((k+2))
        pot[1:-1] = 1/2*(x**2)    #1D initial condition

    if dimension==2:
        '''
        Initial wavefunction
        '''
        wfc = np.zeros((k+2,Ny), dtype = np.complex128)

        wfc[1:-1,:] = ((1/np.pi)**(1/2))*np.exp(-(y_mesh**2 + x_mesh**2)/2)+0j
        '''
        potential
        '''
        pot = np.zeros((k+2,Ny))
        pot[1:-1,:] = 1/2*(x_mesh**2+y_mesh**2)

    elif dimension==3:
        '''
        Initial wavefunction
        '''

        wfc = np.zeros((k+2,Ny,Nz), dtype = np.complex128)

        wfc[1:-1,:,:] = ((1/np.pi)**(3/4))*np.exp(-(z_mesh**2+y_mesh**2+x_mesh**2)/2)+0j    #3D initial condition(x_mesh = (128,256,256),y_mesh=(128,256,256))


        '''
        potential
        '''
        pot = np.zeros((k+2,Ny,Nz))

        pot[1:-1,:,:] = 1/2*(x_mesh**2+y_mesh**2+z_mesh**2)   #3D initial condition



    return wfc, pot


def laplacian(psi):
    temp=copy.deepcopy(psi)
    if dimension == 1:
        if rank in (list(range(procs))[:-1]):
            u_0_to_1 = np.array(psi[-2])
            #print(f"proc {rank} sending to {rank+1}", u_0_to_1)
            r1 = comm.Isend([u_0_to_1, MPI.COMPLEX], dest=rank+1)


            # recv data from 1
            data = np.empty(1, dtype=np.complex128)
            req = comm.Irecv([data, MPI.COMPLEX], source=rank+1)
            req.wait()
            r1.wait()
            #print(f"proc {rank} receiving from {rank+1}", data)
            psi[-1] = data
            temp[1:-2] = (psi[2:-1] - 2*psi[1:-2] + psi[:-3]) / dx**2
            temp[-2] = (psi[-1] - 2*psi[-2] + psi[-3]) / dx**2
            # print(rank)

        if rank in (list(range(procs))[1:]):
            u_1_to_0 = np.array(psi[1])
            #print(f"proc {rank} sending to {rank-1}: ", u_1_to_0)
            req= comm.Isend([u_1_to_0, MPI.COMPLEX], dest=rank-1)

            # recv data from 0
            data = np.empty(1, dtype=np.complex128)
            r1 = comm.Irecv([data, MPI.COMPLEX], source=rank-1)
            r1.wait()
            req.wait()
            #print(f"proc {rank} receiving from {rank-1}", data)
            temp[0] = data
            # temp[0], temp[-1] = 0, 0
            temp[2:-1] = (psi[3:] - 2*psi[2:-1] + psi[1:-2]) / dx**2
            temp[1] = (psi[2] - 2*psi[1] + temp[0]) / dx**2
            # print(rank)
    elif dimension == 2:
        if rank in list(range(procs))[:-1]:
            u_0_to_1 = np.array(psi[-2,:])
            #print("proc 0 sending to 1", u_0_to_1)
            r1 = comm.Isend([u_0_to_1, MPI.COMPLEX], dest=rank+1)

            # recv data from 1
            data = np.empty((1,Ny), dtype=np.complex128)
            req = comm.Irecv([data, MPI.COMPLEX], source=rank+1)
            req.wait()
            r1.wait()
            #print("proc 0 receiving from 1", data)
            psi[-1,:] = data
            temp[1:-2,:] = (psi[2:-1,:] - 2*psi[1:-2,:] + psi[:-3,:]) / dx**2
            temp[-2,:] = (psi[-1,:] - 2*psi[-2,:] + psi[-3,:]) / dx**2
            temp[:,1:-1]+=(psi[:,2:]-2*psi[:,1:-1]+psi[:,:-2])/dy**2

        if rank in list(range(procs))[1:]:
            u_1_to_0 = np.array(psi[1,:])

            #print("proc 1 sending to 0: ", u_1_to_0)
            req = comm.Isend([u_1_to_0, MPI.COMPLEX], dest=rank-1)

            # recv data from 0
            data = np.empty((1,Ny), dtype=np.complex128)
            r1 = comm.Irecv([data, MPI.COMPLEX], source=rank-1)
            r1.wait()
            req.wait()
            #print("proc 1 receiving from 0", data)
            temp[0,:] =data
            temp[2:-1,:]= (psi[3:,:]-2*psi[2:-1,:]+psi[1:-2,:])/dx**2
            temp[1,:] = (psi[2,:] -2*psi[1,:]+temp[0,:])/dx**2
            temp[:,1:-1]+=(psi[:,2:]-2*psi[:,1:-1]+psi[:,:-2])/dy**2

    elif dimension == 3:
        if rank in list(range(procs))[:-1]:
            u_0_to_1 = np.array(psi[-2,:,:])
            #print("proc 0 sending to 1", u_0_to_1)
            r1 = comm.Isend([u_0_to_1, MPI.COMPLEX], dest=rank+1)

            # recv data from 1
            data = np.empty((1,Ny,Nz), dtype=np.complex128)
            req = comm.Irecv([data, MPI.COMPLEX], source=rank+1)
            req.wait()
            r1.wait()
            #print("proc 0 receiving from 1", data)
            psi[-1,:,:] = data
            temp[1:-2,:,:]= (psi[2:-1,:,:]-2*psi[1:-2,:,:]+psi[:-3,:,:])/dx**2
            temp[-2,:,:]= (psi[-1,:,:] -2*psi[-2,:,:]+ psi[-3,:,:])/dx**2

            temp[:,1:-1,:]+=(psi[:,2:,:]-2*psi[:,1:-1,:]+psi[:,:-2,:])/dy**2
            temp[:,:,1:-1]+=(psi[:,:,2:]-2*psi[:,:,1:-1]+psi[:,:,:-2])/dz**2

        if rank in list(range(procs))[1:]:
            u_1_to_0 = np.array(psi[1,:,:])

            #print("proc 1 sending to 0: ", u_1_to_0)
            req = comm.Isend([u_1_to_0, MPI.COMPLEX], dest=rank-1)

            # recv data from 0
            data = np.empty((1,Ny,Nz), dtype=np.complex128)
            r1 = comm.Irecv([data, MPI.COMPLEX], source=rank-1)
            r1.wait()
            req.wait()
            #print("proc 1 receiving from 0", data)
            temp[0,:,:] =data
            temp[2:-1,:,:]= (psi[3:,:,:]-2*psi[2:-1,:,:]+psi[1:-2,:,:])/dx**2
            temp[1,:,:] = (psi[2,:,:] -2*psi[1,:,:]+temp[0,:,:])/dx**2
            temp[:,1:-1,:]+=(psi[:,2:,:]-2*psi[:,1:-1,:]+psi[:,:-2,:])/dy**2
            temp[:,:,1:-1]+=(psi[:,:,2:]-2*psi[:,:,1:-1]+psi[:,:,:-2])/dz**2

    return temp

def integral(psi):
    return volume * np.sum(psi)

def norm(psi):
  return integral(np.abs(psi)**2)


def energy(psi, pot):
    z=np.conjugate(psi)*(-1/2*laplacian(psi)+(pot+g*np.abs(psi)**2/2)*psi)

    return integral(z.real)

def compute_RHS(psi, pot):
    return -1j*(-1/2*laplacian(psi)+(pot + g * np.abs(psi)**2)*psi)

def sstep_RK4(psi, pot):
    k1 = compute_RHS(psi, pot)
    k2 = compute_RHS(psi + k1 * dt/2, pot)
    k3 = compute_RHS(psi + k2 * dt/2, pot)
    k4 = compute_RHS(psi + k3 * dt, pot)
    return psi + (k1 + 2*k2 + 2*k3 + k4) * dt/6


wfc, pot = set_initialcond()

j=0
t_temp = 0
if rank == 0:
    start_time = time.time()
for i in range(nstep):
    #print("i = ", i)
    wfc = sstep_RK4(wfc, pot)
    #if i % 1 == 0:
        #print('\n----------------------')
        #print('t: ', t_temp)
        #j=j+1
    t_temp =t_temp + dt
local_sum = norm(wfc)
total = np.zeros_like(local_sum)

#print("local_sum =", rank, local_sum)
#comm.Reduce(local_sum, total, root=0, op=MPI.SUM)

#print("rank = ", rank, " Total = ", total)



if rank == 0:
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

Overwriting MPI.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 python3 MPI.py

Execution time: 6.431137323379517 seconds
